In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import cv2 as cv
from tqdm import tqdm
import os
import json

from lac.utils.plotting import plot_path_3d, plot_3d_points
from lac.utils.frames import invert_transform_mat

%load_ext autoreload
%autoreload 2

In [ ]:
import symforce

try:
    symforce.set_epsilon_to_symbol()
except symforce.AlreadyUsedEpsilon:
    print("Already set symforce epsilon")
    pass
import symforce.symbolic as sf

from lac.localization.symforce_util import odometry_lander_relpose_fgo
from lac.localization.imu_recovery import recover_rotation_exact, recover_translation

In [ ]:
# data_path = "../../output/imu_20hz/" + "data_log.json"
data_path = "../../results/runs/map1_seed4_spiral_4.5_2.0_run2/data_log.json"
json_data = json.load(open(f"{data_path}"))

poses = []
imu_data = []

for frame in json_data["frames"]:
    poses.append(np.array(frame["pose"]))
    imu_data.append(np.array(frame["imu"]))

imu_data = np.array(imu_data)
initial_pose = np.array(json_data["initial_pose"])
lander_pose_rover = np.array(json_data["lander_pose_rover"])
lander_pose_world = initial_pose @ lander_pose_rover

gt_translations = np.zeros((len(poses), 3))
for i in range(len(poses)):
    gt_translations[i] = poses[i][:3, 3]

dt = 0.05
times = np.arange(0, len(imu_data) * dt, dt)

In [ ]:
R_prev = initial_pose[:3, :3]
t_prev_prev = initial_pose[:3, 3]
t_prev = initial_pose[:3, 3]

Integrate IMU


In [ ]:
imu_rotations = []
imu_rotations.append(R_prev)

imu_translations = []
imu_translations.append(t_prev)

for i in tqdm(range(1, len(poses))):
    # Rotation recovery
    omega = imu_data[i, 3:]
    R_curr = recover_rotation_exact(R_prev, omega, dt)
    imu_rotations.append(R_curr)
    R_prev = R_curr

    # Translation recovery
    a = imu_data[i, :3]
    t_curr = recover_translation(t_prev_prev, t_prev, R_curr, a, dt)
    imu_translations.append(t_curr)
    t_prev_prev = t_prev
    t_prev = t_curr

In [ ]:
delta_rotations = []
delta_translations = []

for i in range(1, len(poses)):
    delta_rotations.append(imu_rotations[i] @ imu_rotations[i - 1].T)
    delta_translations.append(imu_translations[i] - imu_translations[i - 1])

In [ ]:
lander_position = lander_pose_world[:3, 3]

# Lander range and bearing measurements
lander_pose_measurements = []  # relative pose of lander in rover frame
lander_range_measurements = []
lander_los_measurements = []

for i in range(len(poses)):
    t_i = poses[i][:3, 3]
    delta_t = t_i - lander_position
    # TODO: add noise
    lander_range_measurements.append(np.linalg.norm(delta_t))
    lander_pose_measurements.append(invert_transform_mat(poses[i]) @ lander_pose_world)
    lander_los_measurements.append(-delta_t / np.linalg.norm(delta_t))

In [ ]:
ODOM_R_SIGMA = 0.1  # for rotations
ODOM_T_SIGMA = 0.1  # for translations [m]
ODOM_SIGMA = np.ones(6)
ODOM_SIGMA[:3] *= ODOM_R_SIGMA
ODOM_SIGMA[3:] *= ODOM_T_SIGMA

LANDER_ANGLE_SIGMA = 0.0001  # [rad]

LANDER_RELPOSE_SIGMA = 0.01 * np.ones(6)

imu_rollout_poses = []
imu_odometry = []
for i in range(len(poses)):
    pose = np.eye(4)
    pose[:3, :3] = imu_rotations[i]
    pose[:3, 3] = imu_translations[i]
    imu_rollout_poses.append(pose)

for i in range(len(poses) - 1):
    imu_odom = np.eye(4)
    imu_odom[:3, :3] = delta_rotations[i]
    imu_odom[:3, 3] = delta_translations[i]
    imu_odometry.append(imu_odom)

gt_odometry = []
for i in range(len(poses) - 1):
    delta_pose = np.linalg.inv(poses[i]) @ poses[i + 1]
    gt_odometry.append(delta_pose)

In [ ]:
N_WINDOW = 40
N_SHIFT = 10
N = 10000


def sliding_window_fgo():
    init_poses = imu_rollout_poses[:N_WINDOW]
    fgo_poses = [None] * N
    k_max = (N - N_WINDOW) // N_SHIFT

    for k in (pbar := tqdm(range(k_max))):
        window = slice(N_SHIFT * k, N_SHIFT * k + N_WINDOW)
        odometry = imu_odometry[window][:-1]
        # odometry = gt_odometry[window][:-1]
        lander_measurements = lander_pose_measurements[window]

        opt_poses, result = odometry_lander_relpose_fgo(
            init_poses,
            lander_pose_world,
            odometry,
            lander_measurements,
            ODOM_SIGMA,
            LANDER_RELPOSE_SIGMA,
            debug=False,
        )
        fgo_poses[N_SHIFT * k : N_SHIFT * (k + 1)] = opt_poses[:N_SHIFT]

        init_poses[:-N_SHIFT] = opt_poses[N_SHIFT:]
        if k != k_max - 1:
            pose = opt_poses[-1]
            for i in range(N_SHIFT):
                init_poses[-N_SHIFT + i] = pose @ imu_odometry[window][-1]
                pose = init_poses[-N_SHIFT + i]

    return fgo_poses

In [ ]:
fgo_poses = sliding_window_fgo()

In [ ]:
# all poses not None in fgo_poses
fgo_poses = [pose for pose in fgo_poses if pose is not None]
N = len(fgo_poses)

fgo_positions = []
for pose in fgo_poses:
    fgo_positions.append(pose[:3, 3])
fgo_positions = np.array(fgo_positions)

imu_positions = []
for pose in imu_rollout_poses:
    imu_positions.append(pose[:3, 3])
imu_positions = np.array(imu_positions)

print("RMSE: ", np.sqrt(np.mean((fgo_positions - gt_translations[:N]) ** 2)))

fig = plot_path_3d(
    fgo_positions[:, 0], fgo_positions[:, 1], fgo_positions[:, 2], color="orange", name="Optimized"
)
fig = plot_path_3d(
    gt_translations[:N, 0],
    gt_translations[:N, 1],
    gt_translations[:N, 2],
    fig=fig,
    color="blue",
    name="Ground truth",
)
fig = plot_path_3d(
    imu_positions[:N, 0],
    imu_positions[:N, 1],
    imu_positions[:N, 2],
    fig=fig,
    color="green",
    name="IMU initialization",
)
fig = plot_3d_points(
    lander_position[None, :], fig=fig, color="red", markersize=3, name="Lander Origin"
)
fig.update_layout(height=700, width=1200, scene_aspectmode="data")
fig.show()